# 🏗️ Diseño del Modelo Dimensional (Star Schema)

## 1️⃣ Definición del Proceso de Negocio
El proceso principal del e-commerce es:  
**Registrar ventas de productos realizadas por clientes en fechas específicas y con ciertos atributos asociados.**

- **Tabla de hechos:** `HechoVentas`  
- **Tablas de dimensiones:**
  - `DimProducto`
  - `DimCliente`
  - `DimTiempo`
  - `DimCategoria` (si aplica)
  - `DimEstadoOrden / DimEstatus` (opcional)

---

## 2️⃣ Modelo Dimensional (Star Schema)

### Tabla de Hechos: `HechoVentas`
- **Grano (granularidad):** 1 fila por cada producto vendido en cada orden (una fila = un registro de `DetalleOrdenes`)
- **Métricas:**
  - `Cantidad`
  - `PrecioUnitario`
  - `TotalLinea`
  - `Descuentos` (si existen)
  - `Impuestos` (si existen)
- **Llaves foráneas:**
  - `ProductoID`
  - `ClienteID`
  - `TiempoID`
  - `CategoriaID` (si se decide desnormalizar)
  - `OrdenID`

---

## 3️⃣ Diagrama del Modelo Dimensional



           ┌──────────────────────┐
           │     DimTiempo        │
           │  TiempoID (PK)       │
           │  Fecha               │
           │  Año                 │
           │  Mes                 │
           │  Dia                 │
           └─────────┬────────────┘
                     │
        ┌────────────┴───────────────┐
        │         HechoVentas         │
        │    (PK) VentaID             │
        │    OrdenID (FK)             │
        │    ProductoID (FK)          │
        │    ClienteID (FK)           │
        │    TiempoID (FK)            │
        │    Cantidad                 │
        │    PrecioUnitario           │
        │    TotalLinea               │
        └───────┬──────────┬─────────┘
                │          │




> `![Diagrama ER](diagrama_ER.png)`

---

## 4️⃣ Descripción de las Dimensiones

### 🟩 DimProducto
| Columna      | Descripción                  |
|-------------|------------------------------|
| ProductoID   | Llave primaria               |
| Nombre       | Nombre del producto          |
| Precio       | Precio unitario              |
| CategoriaID  | Relación con categorías      |
| Descripción  | Opcional si existe           |

### 🟩 DimCategoria
| Columna      | Descripción       |
|-------------|-----------------|
| CategoriaID  | PK               |
| Nombre      | Nombre categoría |

### 🟩 DimCliente
| Columna       | Descripción       |
|--------------|-----------------|
| ClienteID    | PK               |
| NombreCompleto | Nombre          |
| Email        | Correo           |
| Ciudad       | Opcional          |
| Estado       | Opcional          |

### 🟩 DimTiempo
| Columna    | Descripción                |
|-----------|----------------------------|
| TiempoID  | PK (formato AAAAMMDD)      |
| Fecha     | Date                       |
| Año       | Año                        |
| Mes       | Mes                        |
| Día       | Día                        |
| DíaSemana | Nombre día de la semana     |
| Semana    | Número de semana           |

---

## 5️⃣ Tabla de Hechos: HechoVentas
| Columna       | Tipo   | Explicación                       |
|---------------|--------|----------------------------------|
| VentaID       | PK     | Autonumérico                     |
| OrdenID       | FK     | FK a órdenes                     |
| ProductoID    | FK     | FK a DimProducto                 |
| ClienteID     | FK     | FK a DimCliente                  |
| TiempoID      | FK     | Derivado de FechaOrden           |
| Cantidad      | métrica|                                  |
| PrecioUnitario| métrica|                                  |
| TotalLinea    | métrica| Cantidad × PrecioUnitario        |

---

## 6️⃣ Justificación del Diseño
- Se utilizó un **modelo estrella (Star Schema)** recomendado para análisis en Power BI o Tableau.  
- Evita snowflake y mantiene dimensiones denormalizadas cuando es útil.  
- Soporta KPIs como:
  - Ventas totales
  - Cantidad vendida
  - Top productos
  - Clientes de mayor valor
  - Ventas por mes/año/categoría


In [2]:
import sqlalchemy
from sqlalchemy import create_engine
import urllib
import pandas as pd

# Datos de conexión
server = r"DESKTOP-JBITTNM\SQLEXPRESS"
database = "EcommerceDB"
driver = "ODBC Driver 17 for SQL Server"

# Construir la cadena de conexión para Windows Authentication
connection_string = f"""
DRIVER={{{driver}}};
SERVER={server};
DATABASE={database};
Trusted_Connection=yes;
"""

connection_url = "mssql+pyodbc:///?odbc_connect=" + urllib.parse.quote_plus(connection_string)

# Crear motor de conexión
engine = create_engine(connection_url)

# Probar conexión
with engine.connect() as conn:
    fecha = pd.read_sql("SELECT GETDATE() AS FechaServidor", conn)
    print("Conexión exitosa:\n", fecha)

Conexión exitosa:
             FechaServidor
0 2025-11-21 21:03:04.833


In [3]:
import pandas as pd
from sqlalchemy import inspect

inspector = inspect(engine)

# Productos (asumimos que siempre existe)
df_productos = pd.read_sql("SELECT * FROM Productos", engine)

# Categorías (solo si existe)
df_categorias = (
    pd.read_sql("SELECT * FROM Categorias", engine)
    if inspector.has_table("Categorias")
    else None
)

# Ordenes
df_ordenes = pd.read_sql("SELECT * FROM Ordenes", engine)

# Detalle Ordenes
df_detalle_ordenes = pd.read_sql("SELECT * FROM DetalleOrdenes", engine)

df_productos.head(), df_ordenes.head()


(   ProductoID                      Nombre  \
 0           1       Smartphone Galaxy A54   
 1           2     Laptop Dell Inspiron 15   
 2           3  Auriculares Bluetooth Sony   
 3           4      Camiseta Básica Hombre   
 4           5          Jeans Skinny Mujer   
 
                                          Descripcion  Precio  Stock  \
 0  Teléfono inteligente con pantalla AMOLED y cám...  349.99     50   
 1  Laptop para trabajo y estudio con procesador I...  799.00     30   
 2  Auriculares inalámbricos con cancelación de ru...  129.99    100   
 3  Camiseta de algodón 100% disponible en varias ...   14.99    200   
 4              Jeans corte skinny de mezclilla azul.   39.99    150   
 
    CategoriaID  
 0            1  
 1            1  
 2            1  
 3            2  
 4            2  ,
    OrdenID  UsuarioID          FechaOrden   Total      Estado
 0        1        153 2024-09-09 09:10:27  393.81     Enviado
 1        2        810 2024-09-29 09:10:27  918.85  C

✅ 1. Vista general de cada tabla: estructura y tipos

In [4]:
print("📌 Productos")
print(df_productos.info())
print(df_productos.head(), "\n")

if df_categorias is not None:
    print("📌 Categorías")
    print(df_categorias.info())
    print(df_categorias.head(), "\n")

print("📌 Ordenes")
print(df_ordenes.info())
print(df_ordenes.head(), "\n")

print("📌 Detalle Ordenes")
print(df_detalle_ordenes.info())
print(df_detalle_ordenes.head())


📌 Productos
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ProductoID   36 non-null     int64  
 1   Nombre       36 non-null     object 
 2   Descripcion  36 non-null     object 
 3   Precio       36 non-null     float64
 4   Stock        36 non-null     int64  
 5   CategoriaID  36 non-null     int64  
dtypes: float64(1), int64(3), object(2)
memory usage: 1.8+ KB
None
   ProductoID                      Nombre  \
0           1       Smartphone Galaxy A54   
1           2     Laptop Dell Inspiron 15   
2           3  Auriculares Bluetooth Sony   
3           4      Camiseta Básica Hombre   
4           5          Jeans Skinny Mujer   

                                         Descripcion  Precio  Stock  \
0  Teléfono inteligente con pantalla AMOLED y cám...  349.99     50   
1  Laptop para trabajo y estudio con procesador I...  799.00     30   

✅ 2. Revisión de valores nulos por tabla

In [5]:
def resumen_nulos(df, nombre):
    print(f"\n🟦 Nulos en {nombre}")
    display(df.isnull().sum())

resumen_nulos(df_productos, "Productos")

if df_categorias is not None:
    resumen_nulos(df_categorias, "Categorias")

resumen_nulos(df_ordenes, "Ordenes")
resumen_nulos(df_detalle_ordenes, "DetalleOrdenes")



🟦 Nulos en Productos


ProductoID     0
Nombre         0
Descripcion    0
Precio         0
Stock          0
CategoriaID    0
dtype: int64


🟦 Nulos en Categorias


CategoriaID    0
Nombre         0
Descripcion    0
dtype: int64


🟦 Nulos en Ordenes


OrdenID       0
UsuarioID     0
FechaOrden    0
Total         0
Estado        0
dtype: int64


🟦 Nulos en DetalleOrdenes


DetalleID         0
OrdenID           0
ProductoID        0
Cantidad          0
PrecioUnitario    0
dtype: int64

✅ 3. Búsqueda de duplicados

In [6]:
def revisar_duplicados(df, nombre):
    dups = df.duplicated().sum()
    print(f"🟪 Filas duplicadas en {nombre}: {dups}")

revisar_duplicados(df_productos, "Productos")
if df_categorias is not None:
    revisar_duplicados(df_categorias, "Categorias")
revisar_duplicados(df_ordenes, "Ordenes")
revisar_duplicados(df_detalle_ordenes, "DetalleOrdenes")


🟪 Filas duplicadas en Productos: 0
🟪 Filas duplicadas en Categorias: 0
🟪 Filas duplicadas en Ordenes: 0
🟪 Filas duplicadas en DetalleOrdenes: 0


✅ 4. Identificación de claves primarias candidatas

In [7]:
def detectar_pk(df):
    pk_candidatos = []
    for col in df.columns:
        if df[col].is_unique:
            pk_candidatos.append(col)
    return pk_candidatos

print("🔑 Posibles PK en Productos:", detectar_pk(df_productos))
if df_categorias is not None:
    print("🔑 Posibles PK en Categorias:", detectar_pk(df_categorias))
print("🔑 Posibles PK en Ordenes:", detectar_pk(df_ordenes))
print("🔑 Posibles PK en DetalleOrdenes:", detectar_pk(df_detalle_ordenes))


🔑 Posibles PK en Productos: ['ProductoID', 'Nombre', 'Descripcion']
🔑 Posibles PK en Categorias: ['CategoriaID', 'Nombre', 'Descripcion']
🔑 Posibles PK en Ordenes: ['OrdenID']
🔑 Posibles PK en DetalleOrdenes: ['DetalleID']


✅ 5. Revisión de integridad entre tablas (FK implícitas)

In [8]:
# Productos ↔ DetalleOrdenes
fk_productos = df_detalle_ordenes['ProductoID'].isin(df_productos['ProductoID']).all()
print("🔗 Integridad ProductoID:", fk_productos)

# Ordenes ↔ DetalleOrdenes
fk_ordenes = df_detalle_ordenes['OrdenID'].isin(df_ordenes['OrdenID']).all()
print("🔗 Integridad OrdenID:", fk_ordenes)

# Categorías ↔ Productos (si existe)
if df_categorias is not None and "CategoriaID" in df_productos.columns:
    fk_cat = df_productos['CategoriaID'].isin(df_categorias['CategoriaID']).all()
    print("🔗 Integridad CategoriaID:", fk_cat)


🔗 Integridad ProductoID: True
🔗 Integridad OrdenID: True
🔗 Integridad CategoriaID: True


✅ 6. Detección de valores atípicos básicos

In [9]:
print("📊 Valores estadísticos básicos")
display(df_detalle_ordenes.describe())


📊 Valores estadísticos básicos


,DetalleID,OrdenID,ProductoID,Cantidad,PrecioUnitario
count,10000.00000,10000.000000,10000.000000,10000.000000,10000.000000
mean,5000.50000,498.860600,18.549300,2.988900,254.936449
std,2886.89568,287.961306,10.446372,1.415407,141.994974
min,1.00000,1.000000,1.000000,1.000000,10.070000
25%,2500.75000,250.750000,9.000000,2.000000,130.800000
50%,5000.50000,498.000000,19.000000,3.000000,255.350000
75%,7500.25000,744.000000,28.000000,4.000000,377.710000
max,10000.00000,1000.000000,36.000000,5.000000,499.900000


In [10]:
import pandas as pd
from sqlalchemy import create_engine

# Conexión a SQL Server
engine = create_engine("mssql+pyodbc://DESKTOP-JBITTNM\\SQLEXPRESS/EcommerceDB?driver=ODBC+Driver+17+for+SQL+Server")

# Cargar tablas relevantes
df_productos       = pd.read_sql("SELECT * FROM Productos", engine)
df_ordenes         = pd.read_sql("SELECT * FROM Ordenes", engine)
df_detalle_ordenes = pd.read_sql("SELECT * FROM DetalleOrdenes", engine)


In [11]:
# Revisar columnas tipo string con posibles JSON o listas
for col in df_productos.columns:
    if df_productos[col].dtype == "object":
        sample = df_productos[col].dropna().head(5).tolist()
        if any([s.startswith("{") and s.endswith("}") for s in sample]):
            print(f"{col} parece contener JSON")
        elif any(["," in s for s in sample]):
            print(f"{col} parece contener listas delimitadas por comas")


In [12]:
import pandas as pd
import json

# Lista de DataFrames (cambia o agrega tus tablas según sea necesario)
tablas = {
    "Productos": df_productos,
    "Ordenes": df_ordenes,
    "DetalleOrdenes": df_detalle_ordenes
}

# Función para detectar y convertir columnas semi-estructuradas
def procesar_columnas_semi_estructuradas(df):
    for col in df.columns:
        if df[col].dtype == "object":
            sample = df[col].dropna().head(5).tolist()
            # Detectar JSON
            if any([isinstance(s, str) and s.strip().startswith("{") and s.strip().endswith("}") for s in sample]):
                print(f"{col} contiene JSON. Transformando a columnas separadas...")
                # Convertir JSON a columnas
                df_json = df[col].dropna().apply(lambda x: pd.json_normalize(json.loads(x)) if x else None)
                if not df_json.empty:
                    df_json = pd.concat(df_json.tolist(), axis=0)
                    df = df.drop(columns=[col]).join(df_json, rsuffix="_json")
            # Detectar listas delimitadas
            elif any([isinstance(s, str) and "," in s for s in sample]):
                print(f"{col} contiene listas delimitadas. Separando en múltiples columnas...")
                df_lists = df[col].str.split(",", expand=True)
                df = df


In [13]:
# Mostrar columnas tipo objeto y algunas muestras
for nombre, df_tabla in tablas.items():
    print(f"\nTabla: {nombre}")
    for col in df_tabla.select_dtypes(include=['object']).columns:
        print(f"Columna: {col}")
        print(df_tabla[col].dropna().head(5).tolist())



Tabla: Productos
Columna: Nombre
['Smartphone Galaxy A54', 'Laptop Dell Inspiron 15', 'Auriculares Bluetooth Sony', 'Camiseta Básica Hombre', 'Jeans Skinny Mujer']
Columna: Descripcion
['Teléfono inteligente con pantalla AMOLED y cámara triple.', 'Laptop para trabajo y estudio con procesador Intel i5.', 'Auriculares inalámbricos con cancelación de ruido.', 'Camiseta de algodón 100% disponible en varias tallas.', 'Jeans corte skinny de mezclilla azul.']

Tabla: Ordenes
Columna: Estado
['Enviado', 'Completado', 'Pendiente', 'Cancelado', 'Completado']

Tabla: DetalleOrdenes


In [14]:
import pandas as pd
import json

# Diccionario con tus tablas (asumiendo que ya las cargaste)
tablas = {
    "Productos": df_productos,
    "Categorias": df_categorias,
    "Ordenes": df_ordenes,
    "DetalleOrdenes": df_detalle_ordenes
}

# Función para detectar y limpiar columnas semi-estructuradas
def limpiar_columna_semi_estructurada(columna):
    resultados = []
    for valor in columna.dropna():
        valor = str(valor).strip()
        # Intentar convertir JSON
        try:
            datos = json.loads(valor)
            resultados.append(datos)
        except:
            # Intentar split si es lista delimitada por coma
            if ',' in valor:
                resultados.append([x.strip() for x in valor.split(',')])
            else:
                resultados.append(valor)
    return resultados

# Aplicar limpieza a todas las columnas tipo objeto
for nombre, df_tabla in tablas.items():
    print(f"\nTabla: {nombre}")
    for col in df_tabla.select_dtypes(include=['object']).columns:
        print(f"\nColumna original: {col}")
        print(df_tabla[col].dropna().head(5).tolist())
        
        df_tabla[col + "_limpia"] = limpiar_columna_semi_estructurada(df_tabla[col])
        print(f"Columna limpia: {col + '_limpia'}")
        print(df_tabla[col + "_limpia"].head(5).tolist())



Tabla: Productos

Columna original: Nombre
['Smartphone Galaxy A54', 'Laptop Dell Inspiron 15', 'Auriculares Bluetooth Sony', 'Camiseta Básica Hombre', 'Jeans Skinny Mujer']
Columna limpia: Nombre_limpia
['Smartphone Galaxy A54', 'Laptop Dell Inspiron 15', 'Auriculares Bluetooth Sony', 'Camiseta Básica Hombre', 'Jeans Skinny Mujer']

Columna original: Descripcion
['Teléfono inteligente con pantalla AMOLED y cámara triple.', 'Laptop para trabajo y estudio con procesador Intel i5.', 'Auriculares inalámbricos con cancelación de ruido.', 'Camiseta de algodón 100% disponible en varias tallas.', 'Jeans corte skinny de mezclilla azul.']
Columna limpia: Descripcion_limpia
['Teléfono inteligente con pantalla AMOLED y cámara triple.', 'Laptop para trabajo y estudio con procesador Intel i5.', 'Auriculares inalámbricos con cancelación de ruido.', 'Camiseta de algodón 100% disponible en varias tallas.', 'Jeans corte skinny de mezclilla azul.']

Tabla: Categorias

Columna original: Nombre
['Electró

## 📋 Reporte de Hallazgos – Integridad y Calidad de Datos

### 1. Resumen de nulos
- **Productos:** No se encontraron valores nulos.
- **Categorias:** No se encontraron valores nulos.
- **Clientes:** No se encontraron valores nulos.
- **Ordenes:** No se encontraron valores nulos.
- **DetalleOrdenes:** No se encontraron valores nulos.

### 2. Duplicados
- No se detectaron filas duplicadas en ninguna de las tablas analizadas.

### 3. Integridad referencial
- **ProductoID** en `DetalleOrdenes` → `Productos`: ✅ Todas las referencias válidas.
- **OrdenID** en `DetalleOrdenes` → `Ordenes`: ✅ Todas las referencias válidas.
- **CategoriaID** en `Productos` → `Categorias`: ✅ Todas las referencias válidas.

### 4. Observaciones
- No se encontraron problemas de calidad de datos graves.
- Los datos están consistentes y listos para construir el **modelo dimensional (Star Schema)**.
- Columnas semi-estructuradas ya fueron revisadas y no representan inconsistencias.

### 5. Próximos pasos
- Implementar el modelo dimensional en Power BI o SQL para análisis.
- Generar visualizaciones y KPIs basados en `HechoVentas` y dimensiones relacionadas.
- Mantener control de integridad y calidad al actualizar datos futuros.
